# Benötigte Pakete
- Pandas: Für das Schreiben auf csv-Dateien
- Requests: Für das Senden von HTTP-Anfragen
- BeautifulSoup: Für das Suchen in HTML-Dateien

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Dataframe zu den Metadaten erstellen

In [2]:
df = pd.DataFrame(
        columns=[
            "Parkhauskuerzel",
            "Parkplatzname",
            "Durchfahrtshoehe",
            "Aufzug",
            "Ladestation",
            "Gesamtanzahl",
            "Behindertenstellplaetze",
        ]
    )

# Parkplätze definieren

In [3]:
parking_lots = [
        "K01",
        "K02",
        "K03",
        "K04",
        "N02",
        "N03",
        "N05",
        "N06",
        "N07",
        "S01",
        "S02",
        "S03",
        "S04",
        "S05",
        "S06",
        "S07",
        "W01",
        "W02",
        "W03",
        "W04",
    ]

# Funktion zum Scrapen
Diese Funktion nimmt eine Parkplatznummer entgegen und fügt einen Eintrag in den Dataframe ein.

In [4]:
def scrape_meta_data(parking_lot):
    # url der Stadt Karlsruhe
    BASE_URL = "https://web1.karlsruhe.de/service/Parken/detail.php?id="

    try:
        # HTTP-Anfrage mit der URL des bestimmten Parkhauses
        res = requests.get(f"{BASE_URL}{parking_lot}")
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, "html.parser")

            print(f"Scraping {parking_lot}...")

            # Name des Parkhauses: Erstes h1
            title = soup.find("h1").text

            # Durchfahrtshöhe: Zahl neben "Durchfahrtshöhe"
            height = soup.find(string="Durchfahrtshöhe").find_next("td").text.split()[0]

            # Aufzug
            elevator = soup.find(string="Aufzug").find_next("td").text
            # Daten liegen als "ja" oder "nein" vor: Umwandlung in bool
            elevator = True if elevator == "ja" else False

            # Ladestation
            charging_station = soup.find(string="Ladestation").find_next("td").text
            charging_station = True if charging_station == "ja" else False

            # Gesamtanzahl der Stellplätze
            total_spots = (
                soup.find(string="Gesamtzahl der Stellplätze").find_next("td").text
            )

            # Anzahl der Behindertenstellplätze
            disabled_spots = (
                soup.find(string="Behinderten-Stellplätze").find_next("td").text
            )

            # Einfügen eines neuen Datensatzes in das DataFrame
            df.loc[len(df)] = [
                parking_lot,
                title,
                height,
                elevator,
                charging_station,
                total_spots,
                disabled_spots,
            ]

        else:
            print("Error: ", res.status_code)
    except Exception as e:
        print("Error: ", e)

# Metadaten zu allen Parkplätzen sammeln
Wir iterieren über jedes Parkhaus und sammeln Daten

In [5]:
# für jedes Parkhaus
for parking_lot in parking_lots:
    scrape_meta_data(parking_lot) # Metadaten scrapen

Scraping K01...
Scraping K02...
Scraping K03...
Scraping K04...
Scraping N02...
Scraping N03...
Scraping N05...
Scraping N06...
Scraping N07...
Scraping S01...
Scraping S02...
Scraping S03...
Scraping S04...
Scraping S05...
Scraping S06...
Scraping S07...
Scraping W01...
Scraping W02...
Scraping W03...
Scraping W04...


# Dataframe speichern

In [6]:
# in data/meta_data.csv speichern
df.to_csv("../data/meta_data.csv", index=False)